In [16]:
import numpy as np

def projected_newton_eigen(A, x0, max_iter=50, tol=1e-10):
    """
    Newton/PINVIT style eigenvector solver with projection P(A - λI)P.
    Uses column vector form so xk @ xk.T is valid.
    """
    n = A.shape[0]
    
    # Ensure x0 is a column vector (n,1)
    xk = np.array(x0, dtype=float).reshape(n, 1)
    xk /= np.linalg.norm(xk)
    
    # Initial Rayleigh quotient
    lambdak = (xk.T @ A @ xk)[0,0]
    
    for k in range(max_iter):
        # Residual
        rk = A @ xk - lambdak * xk
        
        # Check convergence
        if np.linalg.norm(rk) < tol:
            break
        
        # Projector onto xk⊥
        P = np.eye(n) - (xk @ xk.T)
        
        # Build projected operator
        M = P @ (A - lambdak * np.eye(n)) @ P
        
        # Solve for δx in xk⊥
        try:
            delta_x = np.linalg.solve(M, -rk)
        except np.linalg.LinAlgError:
            raise RuntimeError("Projected operator became singular.")
        
        # Enforce orthogonality explicitly (numerical safeguard)
        delta_x -= xk * (xk.T @ delta_x)
        
        # Update and normalize
        xk = xk + delta_x
        xk /= np.linalg.norm(xk)
        
        # Update eigenvalue
        lambdak = (xk.T @ A @ xk)[0,0]
    
    return lambdak, xk

# ----------------------------
# Example: Test on 2×2 matrix
# ----------------------------
# Construct the 10x10 tridiagonal matrix with eigenvalues 1, 2, ..., 10
n = 10
main_diag = np.arange(1, n+1)  # Diagonal entries: 1, 2, ..., 10
off_diag = np.ones(n-1)         # Off-diagonal entries: 1, 1, ..., 1 (length 9)

# Create the tridiagonal matrix
A = np.diag(main_diag) + np.diag(off_diag, k=1) + np.diag(off_diag, k=-1)
x0 = np.random.rand(n)  # initial guess
lam, x = projected_newton_eigen(A, x0)

print("Approximate eigenvalue:", lam)
print("Approximate eigenvector:", x)

print("Residual norm:", np.linalg.norm(A @ x - lam * x))


Approximate eigenvalue: 6.000217522257098
Approximate eigenvector: [[-0.00678499]
 [-0.03392643]
 [-0.1289281 ]
 [-0.35288591]
 [-0.57692048]
 [-0.22416007]
 [ 0.57687172]
 [-0.35258617]
 [ 0.12822393]
 [-0.03205773]]
Residual norm: 3.6085109317929275e-11
